### **Paso 2.1 - Creamos el Data flow "df_transform_cases_deaths" para transformar los datos del archivo "cases_deaths.csv"**

1. Configurar el stream **Source** y crear un nuevo dataset de origen que haga referencia al archivo **cases_deaths.csv**, llamado  
   **df_raw_cases_and_deaths**

2. Agregar un stream **Filter** para filtrar el campo **continent** y nos devuelva solo registros para el continente **Europe**

3. Agregar un stream **Select** para seleccionar las columnas que necesitamos

4. Agregar un stream **Pivot**

5. Agregar un stream **Lookup** y un stream **Source**. Además, crear un nuevo dataset de origen que haga referencia al archivo **country_lookup.csv**, llamado **ds_country_lookup**

6. Nuevamente, agregar un stream **Select** para seleccionar y ordenar las columnas que necesitamos   

7. Y para finalizar, utilizaremos un stream **Sink** y creamos un un nuevo dataset de destino que haga referencia a la ruta **processed/ecdc/cases_deaths** de nuestro ADLS. No se indicó un nombre de archivo

8. Creamos un pipeline llamado **pl_process_cases_and_deaths_data*** para ejecutar el Data flow recién creado

In [ ]:
          SOURCE                          FILTER                            SELECT                           PIVOT                      LOOKUP                        SELECT                         SINK
 ________________________          ____________________          ____________________________          _________________          ___________________          ___________________          _______________________
|                        |        |                    |        |                            |        |                 |        |                   |        |                   |        |                       |      
|  CasesAndDeathsSource  |--------|  FilterEuropeOnly  |--------|  SelectOnlyRequiredFileds  |--------|   PivotCounts   |--------|   LookupCountry   |--------|   SelectForSink   |--------|   CaseAndDeathsSink   |
|________________________| +      |____________________| +      |____________________________| +      |_________________| +      |___________________| +      |___________________| +      |_______________________|
                                                                                                                                      |    
        SOURCE                                                                                                                        |    
 ___________________                                                                                                                  |    
|                   |-----------------------------------------------------------------------------------------------------------------'    
|   CountryLookup   |        
|___________________|        

#### Configurar el stream **Source** y crear un nuevo dataset de origen que haga referencia al archivo **cases_deaths.csv**, llamado **df_raw_cases_and_deaths**

Creamos un nuevo contenedor **lookup** en ADLS y cargamos el archivo **country_lookup.csv**.

Comenzamos configurando nuestro "stream" **source** que se llama **CasesAndDeathsSource**. Creamos un nuevo dataset de origen que haga referencia al archivo **cases_deaths.csv**, llamado **df_raw_cases_and_deaths**

<center><img src="https://i.postimg.cc/YCYFG4nh/adf781.png"></center>
<center><img src="https://i.postimg.cc/x8BkqYLv/adf782.png"></center>
<center><img src="https://i.postimg.cc/8c7J1sfk/adf783.png"></center>
<center><img src="https://i.postimg.cc/mDX1f7D6/adf784.png"></center>
<center><img src="https://i.postimg.cc/7L42gxdX/adf785.png"></center>
<center><img src="https://i.postimg.cc/bvR2g7jP/adf786.png"></center>
<center><img src="https://i.postimg.cc/zBXHcxJg/adf787.png"></center>
<center><img src="https://i.postimg.cc/gJx5PNPS/adf788.png"></center>

#### Agregar un stream **Filter** para filtrar el campo **continent** y nos devuelva solo registros para el continente **Europe**

En la opción **Filter on** escribimos la siguiente condición para el filtro:
```
continent == 'Europe' && not(isNull(country_code))
```
<center><img src="https://i.postimg.cc/s2r7zXdt/adf789.png"></center>
<center><img src="https://i.postimg.cc/QMJ5d3jV/adf790.png"></center>
<center><img src="https://i.postimg.cc/Tw2by4P0/adf791.png"></center>

#### Agregar un stream **Select** para seleccionar los campos que necesitamos

Primero veamos que nos permite hacer el stream **Select** :

```
1. Add Mapping --> Fixed mapping
```
Nos permite duplicar un campo ya existente. Como tenemos la casilla **Skip duplicate input columns** activada, solo se tomará en cuenta la primera columna que encuentre

<center><img src="https://i.postimg.cc/BbL8W8N9/adf792.png"></center>
<center><img src="https://i.postimg.cc/05FMYf28/adf793.png"></center>

```
2. Add Mapping --> Rule-based mapping
```
a) Podemos utilizar **Delete** para borrar todas las columnas y utilizar **Rule-based mapping**, es decir, utilizar expresiones que indiquen que columnas queremos utilizar

<center><img src="https://i.postimg.cc/Nj7Kwv8M/adf794.png"></center>

Podemos hacer uso de **true()** para llamar a TODAS LAS COLUMNAS. Y podemos actualizar el nombre de las columnas anteponiendo **cases_** para cada una de ellas. Para hacer esto usamos la expresión **'cases_' + $$** 

- Donde **$$** llama al mismo nombre de la columna 

<center><img src="https://i.postimg.cc/XqRrnVDZ/adf795.png"></center>
<center><img src="https://i.postimg.cc/8zV73z69/adf796.png"></center>
<center><img src="https://i.postimg.cc/7YzCm2fS/adf797.png"></center>

b) Podemos hacer uso de **type == 'string'** para llamar SOLO A LAS COLUMNAS DE TIPO STRING. Y podemos actualizar el nombre de las columnas utilizando **$$** para que lleve el mismo nombre y termine con la palabra **_string** 

- Por tanto, utilizamos la expresión **$$ + '_string'**

<center><img src="https://i.postimg.cc/TPthpj4v/adf798.png"></center>

```
Transformación real utilizando el stream "Select"
```
Pulsamos **Reset** para volver al estado original de las columnas

<center><img src="https://i.postimg.cc/63TqR7k2/adf799.png"></center>

Eliminamos 3 columnas: **continent**, **rate_14_day** y **date**

<center><img src="https://i.postimg.cc/cJxrxkTc/adf800.png"></center>

Luego utilizamos un **Rule-based mapping** para volver a llamar al campo **date** (que habiamos eliminado anteriormente) y le actualizamos su nombre utilizando la expresión **'reported' + '_date'**

<center><img src="https://i.postimg.cc/nz5z1Fxd/adf801.png"></center>
<center><img src="https://i.postimg.cc/t7G4LBYW/adf802.png"></center>
<center><img src="https://i.postimg.cc/RhH0skVv/adf803.png"></center>
<center><img src="https://i.postimg.cc/xjct5nML/adf804.png"></center>
<center><img src="https://i.postimg.cc/44qwFqQq/adf805.png"></center>

#### Agregar un stream **Pivot**

In [ ]:
Esto es lo que realizamos con la tarea PIVOT:

1.- Agrupamos columnas 
 _______________________________________________                                       _________________________________________________________________ 
|country            country_code    population  |    indicator           daily_count  |   reported_date     source                                      |
|United Kingdom     GBR             66647112    |    confirmed cases     0            |   2020-01-02        Epidemic intelligence, national daily data  |           
|United Kingdom     GBR             66647112    |    deaths              0            |   2020-01-02        Epidemic intelligence, national daily data  |
|United Kingdom     GBR             66647112    |    confirmed cases     0            |   2020-01-03        Epidemic intelligence, national daily data  |
|United Kingdom     GBR             66647112    |    deaths              0            |   2020-01-03        Epidemic intelligence, national daily data  |
|United Kingdom     GBR             66647112    |    confirmed cases     0            |   2020-01-04        Epidemic intelligence, national daily data  |
|United Kingdom     GBR             66647112    |    deaths              0            |   2020-01-04        Epidemic intelligence, national daily data  |
|_______________________________________________|                                     |_________________________________________________________________|

2.- Elegimos una columna PIVOT KEY: indicator

3.- Elegimos la(s) columna PIVOTEADA: daily_count
 __________________________________________________________________________________________________________________________________________________________
|country            country_code    population      reported_date     source                                        confirmed cases_count   deaths_count   |                   
|United Kingdom     GBR             66647112        2020-01-02        Epidemic intelligence, national daily data    0                       0              | 
|United Kingdom     GBR             66647112        2020-01-03        Epidemic intelligence, national daily data    0                       0              | 
|United Kingdom     GBR             66647112        2020-01-04        Epidemic intelligence, national daily data    0                       0              | 
|__________________________________________________________________________________________________________________________________________________________|

Visualización de los datos que tenemos hasta el momento

<center><img src="https://i.postimg.cc/G2HGFP9G/adf806.png"></center>
<center><img src="https://i.postimg.cc/HxR5BJ7K/adf807.png"></center>

Agrupamos las columnas: **country**, **country_code**, **population**, **source** y **reported_date**. Creamos la columna **country_special** que unirá los valores de las columnas **country** y **country_code** (esta columna es solo de ejemplo, se borarrá al final)

<center><img src="https://i.postimg.cc/ZRBvBTDL/adf808.png"></center>

La columna **Pivot key** sera **indicator**

<center><img src="https://i.postimg.cc/ydMg0wMC/adf809.png"></center>

La **columna pivoteada** será **daily_count**. Como **expression prefix** colocamos **count**

<center><img src="https://i.postimg.cc/cLhnPknf/adf813.png"></center>
<center><img src="https://i.postimg.cc/wBBmz5XZ/adf814.png"></center>

Previsualizamos lo que llevamos hasta el momento

<center><img src="https://i.postimg.cc/K8xTqhRZ/adf811.png"></center>
<center><img src="https://i.postimg.cc/SKtzxXVs/adf815.png"></center>
<center><img src="https://i.postimg.cc/pXt5rMQD/adf816.png"></center>
<center><img src="https://i.postimg.cc/15cnN7Mb/adf817.png"></center>

Como se comentó al inicio, eliminaremos la columna **country_special**

<center><img src="https://i.postimg.cc/DfGb7wGv/adf818.png"></center>
<center><img src="https://i.postimg.cc/yx0JjLKg/adf819.png"></center>

#### Agregar un stream **Lookup**

Comenzamos creando un nuevo dataset de origen que haga referencia al archivo **country_lookup.csv**, llamado **ds_country_lookup**. Se encuentra en la ruta **lookup/country_lookup.csv**, siendo **lookup** el contenedor

<center><img src="https://i.postimg.cc/gJgLrgp3/adf820.png"></center>
<center><img src="https://i.postimg.cc/kMZDZ6gM/adf821.png"></center>
<center><img src="https://i.postimg.cc/tTv797d5/adf822.png"></center>
<center><img src="https://i.postimg.cc/fRQyzGd6/adf823.png"></center>

En un nuevo stream **Source** llamado **CountryLookup** nos conectamos al dataset recien creado **ds_country_lookup**

<center><img src="https://i.postimg.cc/br2Y8gwy/adf824.png"></center>
<center><img src="https://i.postimg.cc/3w7kBsJg/adf825.png"></center>
<center><img src="https://i.postimg.cc/qqtqq0jd/adf826.png"></center>

Agregamos el stream **Lookup** y nos conectamos al stream **Source** : **CountryLookup**

<center><img src="https://i.postimg.cc/4nkf3cYz/adf827.png"></center>
<center><img src="https://i.postimg.cc/RhjFn8DJ/adf828.png"></center>
<center><img src="https://i.postimg.cc/sD7S494C/adf829.png"></center>
<center><img src="https://i.postimg.cc/2SYWcb0b/adf830.png"></center>

#### Agregar un stream **Select**

De esta manera seleccionamos y ordenamos las columnas que vamos a utilizar

<center><img src="https://i.postimg.cc/7h7CXw7n/adf831.png"></center>
<center><img src="https://i.postimg.cc/s26Zgns7/adf832.png"></center>
<center><img src="https://i.postimg.cc/ydD3gF85/adf833.png"></center>

### Agregar un stream **Sink**

Creamos un nuevo contenedor de nombre **processed** en ADLS

Creamos un nuevo dataset de destino **ds_processed_cases_and_deaths** que haga referencia a la ruta **processed/ecdc/cases_deaths**. El directorio **ecdc** y el subdirectorio **cases_deaths** no existen, por lo que se crearan de manera automática. No se indicó un nombre de archivo

<center><img src="https://i.postimg.cc/NGPgN5xs/adf834.png"></center>
<center><img src="https://i.postimg.cc/mDpG65CX/adf835.png"></center>
<center><img src="https://i.postimg.cc/3RxT1T40/adf836.png"></center>
<center><img src="https://i.postimg.cc/MGDw1TT8/adf837.png"></center>

Nos conectamos al dataset recien creado **ds_processed_cases_and_deaths**

<center><img src="https://i.postimg.cc/6Qww7RB0/adf838.png"></center>
<center><img src="https://i.postimg.cc/TYLTBfkr/adf839.png"></center>
<center><img src="https://i.postimg.cc/Yq5tHZVT/adf840.png"></center>
<center><img src="https://i.postimg.cc/Z564Kjzn/adf841.png"></center>

#### Creamos un pipeline para ejecutar el dataflow

Creamos un pipeline llamado **pl_process_cases_and_deaths_data*** para ejecutar el Data flow recién creado

<center><img src="https://i.postimg.cc/tJgv8RpW/adf842.png"></center>

**Validamos**, **Publicamos** y ejecutamos el pipeline utilizando **Debug**. Recordar que no indicamos ningún archivo de salida, por tanto, nos devolverá un archivo con múltiples particiones

<center><img src="https://i.postimg.cc/SRt140x5/adf843.png"></center>
<center><img src="https://i.postimg.cc/63WMG67g/adf844.png"></center>

En la pestaña de **Settings** indicamos que queremos que SEA UN SOLO ARCHIVO DE SALIDA y le damos un nombre al archivo

<center><img src="https://i.postimg.cc/FKgPRkK0/adf845.png"></center>
<center><img src="https://i.postimg.cc/SN31tp5Y/adf846.png"></center>
<center><img src="https://i.postimg.cc/hGqyc1KS/adf847.png"></center>







